[YOLOv8 of roboflow](https://colab.research.google.com/github/QIN2DIM/hcaptcha-challenger/blob/main/automation/roboflow_yolov8.ipynb)

In [1]:
!nvidia-smi

Thu Mar 14 10:51:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -U onnx onnxruntime-gpu onnxruntime
!pip install ultralytics==8.0.202

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

Paste roboflow snippet into here ↓

In [ ]:
# !pip install roboflow
#
# from roboflow import Roboflow
# rf = Roboflow(api_key="APIKEY")
# project = rf.workspace("WORKSPACE").project("PROJECT")
# dataset = project.version(1).download("yolov8")

Train

In [ ]:
import os
import shutil
from pathlib import Path

from ultralytics import YOLO

HOME = os.getcwd()
%cd {HOME}

challenge_dir_name = "animal-head-2310-1"
output_model_name = "head_of_the_animal_2311_yolov8s.onnx"

this_dir = Path(os.path.abspath("__file__")).parent
yolo_data_dir = this_dir.joinpath(challenge_dir_name)
yolo_inspect_path = yolo_data_dir.joinpath("data.yaml")

model = YOLO("yolov8s.pt")  # detection
# model = YOLO("yolov8s-seg.pt") # segment
model.train(data=str(yolo_inspect_path), epochs=300, imgsz=640)

Export YOLOv8 to ONNX

In [ ]:
# detection
yolo_best_pt_path = this_dir.joinpath("runs/detect/train/weights/best.pt")
yolo_onnx_path = this_dir.joinpath("runs/detect/train/weights/best.onnx")

# segment
# yolo_best_pt_path = this_dir.joinpath("runs/segment/train/weights/best.pt")
# yolo_onnx_path = this_dir.joinpath("runs/segment/train/weights/best.onnx")

model_path = this_dir.joinpath(output_model_name)

# export to onnx
YOLO(str(yolo_best_pt_path)).export(format="onnx", simplify=True, int8=True)

# copy to current work dir
shutil.copyfile(yolo_onnx_path, model_path)

Deployment model to GitHub

In [ ]:
import locale

locale.getpreferredencoding = lambda d=True: "UTF-8"

!pip install PyGithub

from github import Auth, Github, GithubException

auth = Auth.Token(os.getenv("GITHUB_TOKEN", ""))
repo = Github(auth=auth).get_repo("QIN2DIM/hcaptcha-challenger")
modelhub_title = "ONNX ModelHub"


def quick_development(pending_onnx_path: Path):
    if not pending_onnx_path.exists() or not pending_onnx_path.name.endswith(".onnx"):
        return

    for release in repo.get_releases():
        if release.title != modelhub_title:
            continue
        try:
            res = release.upload_asset(path=str(pending_onnx_path))
        except GithubException as err:
            if err.status == 422:
                print(
                    "The model file already exists, please manually replace the file with the same"
                    f" name - url={repo.releases_url}"
                )
        except Exception as err:
            print(err)
        else:
            print(
                f"Model file uploaded successfully - name={res.name} url={res.browser_download_url}"
            )


quick_development(pending_onnx_path=model_path)